In [1]:
!pip install datasets==1.18.4

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import re
from bs4 import BeautifulSoup
import nltk
from nltk.corpus import stopwords

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import f1_score, accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from transformers import AutoTokenizer, AutoModel, DistilBertForSequenceClassification, DistilBertConfig, Trainer, TrainingArguments
from transformers import get_cosine_schedule_with_warmup
from datasets import load_metric

from torch.optim import AdamW

import warnings
warnings.filterwarnings("ignore")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 312.1/312.1 kB 6.7 MB/s eta 0:00:00
  Attempting uninstall: datasets
    Found existing installation: datasets 3.0.1
    Uninstalling datasets-3.0.1:
      Successfully uninstalled datasets-3.0.1


In [2]:
df = pd.read_csv('/kaggle/input/imdb-dataset-of-50k-movie-reviews/IMDB Dataset.csv')
df['sentiment'] = df['sentiment'].map({'positive': 1, 'negative': 0})
print(df.head(5))

                                              review  sentiment
0  One of the other reviewers has mentioned that ...          1
1  A wonderful little production. <br /><br />The...          1
2  I thought this was a wonderful way to spend ti...          1
3  Basically there's a family where a little boy ...          0
4  Petter Mattei's "Love in the Time of Money" is...          1


In [3]:
def missing_values_analysis(df):
    na_columns_ = [col for col in df.columns if df[col].isnull().sum() > 0]
    n_miss = df[na_columns_].isnull().sum().sort_values(ascending=True)
    ratio_ = (df[na_columns_].isnull().sum() / df.shape[0] * 100).sort_values(ascending=True)
    missing_df = pd.concat([n_miss, np.round(ratio_, 2)], axis=1, keys=['Total Missing Values', 'Ratio'])
    missing_df = pd.DataFrame(missing_df)
    return missing_df

In [4]:
import spacy
from nltk.corpus import stopwords
import re
from bs4 import BeautifulSoup

# Load spaCy's small English model
nlp = spacy.load("en_core_web_sm")

# Initialize stopwords
stop_words = set(stopwords.words("english"))

def process(review):
    # Remove HTML tags
    review = BeautifulSoup(review, "html.parser").get_text()
    
    # Remove non-alphabetical characters (e.g., numbers, punctuation)
    review = re.sub(r"[^a-zA-Z]", ' ', review)
    
    # Lowercase the text
    review = review.lower()

    review = re.sub(r'\s+', ' ', review).strip()
    
    # Tokenization and Lemmatization with spaCy
    doc = nlp(review)
    review = [token.lemma_ for token in doc if token.text not in stop_words]
    
    return " ".join(review)

train_data = []

for i in range(len(df["review"])):
    if (i+1) % 2500 == 0:
        print("Processed reviews:", i+1)
    
    train_data.append((df["review"][i]))
    # train_data.append(process(df["review"][i]))

Processed reviews: 2500
Processed reviews: 5000
Processed reviews: 7500
Processed reviews: 10000
Processed reviews: 12500
Processed reviews: 15000
Processed reviews: 17500
Processed reviews: 20000
Processed reviews: 22500
Processed reviews: 25000
Processed reviews: 27500
Processed reviews: 30000
Processed reviews: 32500
Processed reviews: 35000
Processed reviews: 37500
Processed reviews: 40000
Processed reviews: 42500
Processed reviews: 45000
Processed reviews: 47500
Processed reviews: 50000


In [5]:
train_texts, remaining_texts, train_labels, remaining_labels = train_test_split(
    df['review'].tolist(),
    df['sentiment'].tolist(),
    test_size=0.2,
    random_state=42
)

val_texts, test_texts, val_labels, test_labels = train_test_split(
    remaining_texts,
    remaining_labels,
    test_size=0.2,
    random_state=42
)

print(f"Training set size: {len(train_texts)}")
print(f"Validation set size: {len(val_texts)}")
print(f"Test set size: {len(test_texts)}")

Training set size: 40000
Validation set size: 8000
Test set size: 2000


In [6]:
from transformers import AutoModel, AutoTokenizer, Trainer, TrainingArguments
from datasets import load_dataset
import torch
import torch.nn as nn
import torch.nn.functional as F

import wandb
wandb.init(mode='disabled')

simcse_model = AutoModel.from_pretrained("princeton-nlp/sup-simcse-bert-base-uncased")
simcse_tokenizer = AutoTokenizer.from_pretrained("princeton-nlp/sup-simcse-bert-base-uncased")

class ContrastiveLoss(nn.Module):
    def __init__(self, temperature=0.05):
        super(ContrastiveLoss, self).__init__()
        self.temperature = temperature

    def forward(self, anchor, positive):
        sim = F.cosine_similarity(anchor, positive)
        sim = sim / self.temperature
        loss = -torch.log(torch.exp(sim) / torch.sum(torch.exp(sim), dim=-1))
        return loss.mean()

class SimCSETrainer(Trainer):
    def __init__(self, *args, contrastive_loss_fn, **kwargs):
        super().__init__(*args, **kwargs)
        self.contrastive_loss_fn = contrastive_loss_fn

    def compute_loss(self, model, inputs, return_outputs=False):
        input_ids = inputs["input_ids"]
        attention_mask = inputs["attention_mask"]
        
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        embeddings = outputs.pooler_output
        anchor = embeddings[::2]
        positive = embeddings[1::2]
        
        loss = self.contrastive_loss_fn(anchor, positive)

        return (loss, outputs) if return_outputs else loss

    def prediction_step(self, model, inputs, prediction_loss_only=False, ignore_keys=None):
        if "labels" in inputs:
            del inputs["labels"]
        
        return super().prediction_step(model, inputs, prediction_loss_only, ignore_keys)

# Load dataset
dataset = load_dataset("imdb", split="train[:10%]")
dataset = dataset.map(lambda x: simcse_tokenizer(x['text'], padding=True, truncation=True, max_length=512), batched=True)

training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=1,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=200,
    evaluation_strategy="steps",
    save_strategy="steps",
)

train_data = load_dataset("imdb", split="train[:90%]")
eval_data = load_dataset("imdb", split="train[90%:]")

train_data = train_data.map(lambda x: simcse_tokenizer(x['text'], padding=True, truncation=True, max_length=512), batched=True)
eval_data = eval_data.map(lambda x: simcse_tokenizer(x['text'], padding=True, truncation=True, max_length=512), batched=True)

trainer = SimCSETrainer(
    model=simcse_model,
    args=training_args,
    train_dataset=train_data,
    eval_dataset=eval_data,
    contrastive_loss_fn=ContrastiveLoss(temperature=0.05),
)

trainer.train()


config.json:   0%|          | 0.00/689 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/252 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

...

  0%|          | 0/23 [00:00<?, ?ba/s]

  0%|          | 0/3 [00:00<?, ?ba/s]

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


Step,Training Loss,Validation Loss
200,3.136000,No log
400,2.773100,No log
600,2.772800,No log


TrainOutput(global_step=704, training_loss=2.873091784390536, metrics={'train_runtime': 1452.1734, 'train_samples_per_second': 15.494, 'train_steps_per_second': 0.485, 'total_flos': 5919892439040000.0, 'train_loss': 2.873091784390536, 'epoch': 1.0})

In [7]:
train_encodings = simcse_tokenizer(train_texts, truncation=True, padding=True, max_length=512, return_tensors="pt")
val_encodings = simcse_tokenizer(val_texts, truncation=True, padding=True, max_length=512, return_tensors="pt")

In [8]:
class SimCSEMovieReviewDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = SimCSEMovieReviewDataset(train_encodings, train_labels)
val_dataset = SimCSEMovieReviewDataset(val_encodings, val_labels)

In [9]:
from transformers import Trainer, TrainingArguments, EarlyStoppingCallback, DataCollatorWithPadding

model = DistilBertForSequenceClassification.from_pretrained(
    "distilbert-base-uncased-finetuned-sst-2-english", num_labels=2
)

optimizer = AdamW(
    model.parameters(),
    lr=2e-5
)

lr_scheduler = get_cosine_schedule_with_warmup(
    optimizer,
    num_warmup_steps=100,
    num_training_steps=len(train_texts) * 2
)

config = DistilBertConfig.from_pretrained(
    "distilbert-base-uncased-finetuned-sst-2-english", 
    num_labels=2, 
    dropout=0.2,
    attention_dropout=0.2
)

training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="steps",
    save_strategy="steps",
    logging_dir="./logs",
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=2,
    num_train_epochs=5,
    warmup_steps=200,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    eval_steps=500,
    logging_steps=500,
    learning_rate=1e-5,
    weight_decay=0.01,
)

accuracy_metric = load_metric("accuracy")
f1_metric = load_metric("f1")
precision_metric = load_metric("precision")
recall_metric = load_metric("recall")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    accuracy = accuracy_metric.compute(predictions=predictions, references=labels)
    f1 = f1_metric.compute(predictions=predictions, references=labels, average="weighted")
    precision = precision_metric.compute(predictions=predictions, references=labels, average="weighted")
    recall = recall_metric.compute(predictions=predictions, references=labels, average="weighted")
    return {
        "accuracy": accuracy['accuracy'],
        "f1": f1['f1'],
        "precision": precision['precision'],
        "recall": recall['recall']
    }

early_stopping = EarlyStoppingCallback(early_stopping_patience=2)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
    optimizers=(optimizer, lr_scheduler),
    callbacks=[early_stopping]
)

config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.41k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.06k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.09k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.09k [00:00<?, ?B/s]

In [10]:
# Huấn luyện mô hình
trainer.train()

A ConfigError was raised whilst setting the number of model parameters in Weights & Biases config.


Step,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
500,0.271500,0.247461,0.911500,0.911490,0.911569,0.911500
1000,0.232900,0.219658,0.917625,0.917610,0.918250,0.917625
1500,0.221700,0.223243,0.915375,0.915266,0.916968,0.915375
2000,0.210300,0.205367,0.923625,0.923582,0.924234,0.923625
2500,0.211500,0.200807,0.925625,0.925609,0.926344,0.925625
3000,0.138000,0.251382,0.931125,0.931108,0.931343,0.931125
3500,0.141200,0.280599,0.926500,0.926376,0.928737,0.926500
4000,0.136300,0.246297,0.931625,0.931601,0.932663,0.931625
4500,0.140500,0.205602,0.931000,0.931001,0.931008,0.931000
5000,0.151500,0.229890,0.931250,0.931225,0.931619,0.931250


TrainOutput(global_step=5000, training_loss=0.18554462738037109, metrics={'train_runtime': 3776.8754, 'train_samples_per_second': 52.954, 'train_steps_per_second': 3.31, 'total_flos': 1.059739189248e+16, 'train_loss': 0.18554462738037109, 'epoch': 2.0})

In [11]:
def predict_sentiment(text):
    inputs = simcse_tokenizer(text, return_tensors='pt', truncation=True, padding=True, max_length=512)
    
    inputs = {key: value for key, value in inputs.items() if key != 'token_type_ids'}

    for key in inputs:
        inputs[key] = inputs[key].to(model.device)

    outputs = model(**inputs)
    prediction = torch.argmax(outputs.logits, dim=-1)
    return "Positive" if prediction.item() == 1 else "Negative"

In [12]:
sentence = "So hot today =_=  don`t like it and i hate my new timetable, having such a bad week"
print("Original sentence: " + sentence)
print("Predict before: " + predict_sentiment(sentence))
sentence = process(sentence)
print("After processing sentence: " + sentence)
print("Predict after: " + predict_sentiment(sentence))

Original sentence: So hot today =_=  don`t like it and i hate my new timetable, having such a bad week
Predict before: Negative
After processing sentence: hot today like hate new timetable bad week
Predict after: Negative


In [13]:
correct_predictions = sum(
    1 for i in range(len(test_texts))
    if (predict_sentiment(process(test_texts[i])) == "Positive") == test_labels[i]
)

accuracy = correct_predictions / len(test_labels)
print(accuracy)

0.787
